In [1]:
# ------------------------------------------------------------------------
#
# TITLE - fit_dens_gse_v2_ksf.ipynb
# AUTHOR - James Lane
# PROJECT - ges-mass
#
# ------------------------------------------------------------------------
#
# Docstrings and metadata:
'''Take the best-fitting models from the GSE v1 KSF analysis and fit them 
with the v2 KSF.
'''

__author__ = "James Lane"

In [2]:
### Imports

# Basic
import os, sys, copy, dill as pickle
import numpy as np

# Matplotlib and plotting 
import matplotlib.pyplot as plt

# Project specific
sys.path.insert(0,'../../src/')
from ges_mass import mass as pmass
from ges_mass import densprofiles as pdens
from ges_mass import util as putil

### Notebook setup

%matplotlib inline
plt.style.use('../../src/mpl/project.mplstyle') # This must be exactly here
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

A new version of galpy (1.8.2) is available, please upgrade using pip/conda/... to get the latest features and bug fixes!
/home/lane/software/galpy/galpy/util/bovy_plot.py:7: FutureWarning: galpy.util.bovy_plot is being deprecated in favor of galpy.util.plot; functions inside of this module have also changed name, but all functions still exist; please switch to the new import and new function names, because the old import and function names will be removed in v1.9
  warnings.warn('galpy.util.bovy_plot is being deprecated in favor of galpy.util.plot; functions inside of this module have also changed name, but all functions still exist; please switch to the new import and new function names, because the old import and function names will be removed in v1.9',FutureWarning)

/home/lane/software/miniconda3/envs/phd/lib/python3.10/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 1. is an invalid version and will not be supported in a future release
  warnings.warn

### Keywords, Pathing, Loading, Data Preparation

In [3]:
# %load ../../src/nb_modules/keywords_pathing_loading_data_prep.py
## Keywords
cdict = putil.load_config_to_dict()
keywords = ['BASE_DIR','APOGEE_DR','APOGEE_RESULTS_VERS','GAIA_DR','NDMOD',
            'DMOD_MIN','DMOD_MAX','LOGG_MIN','LOGG_MAX','FEH_MIN','FEH_MAX',
            'FEH_MIN_GSE','FEH_MAX_GSE','DF_VERSION','KSF_VERSION','NPROCS',
            'RO','VO','ZO']
base_dir,apogee_dr,apogee_results_vers,gaia_dr,ndmod,dmod_min,dmod_max,\
    logg_min,logg_max,feh_min,feh_max,feh_min_gse,feh_max_gse,df_version,\
    ksf_version,nprocs,ro,vo,zo = putil.parse_config_dict(cdict,keywords)
logg_range = [logg_min,logg_max]
feh_range = [feh_min,feh_max]
feh_range_gse = [feh_min_gse,feh_max_gse]
feh_range_all = [feh_min,feh_max_gse]
feh_range_fit = copy.deepcopy(feh_range_gse) # Need to choose here

df_version = 'v2_lb_ps_hb_tp'
ksf_version = 'v2.03_beta_03_6e8_09_1.5e8'

## Pathing
fit_paths = putil.prepare_paths(base_dir,apogee_dr,apogee_results_vers,gaia_dr,
                                df_version,ksf_version)
data_dir,version_dir,ga_dir,gap_dir,df_dir,ksf_dir,fit_dir = fit_paths

## Filenames
fit_filenames = putil.prepare_filenames(ga_dir,gap_dir,feh_range_fit)
apogee_SF_filename,apogee_effSF_filename,apogee_effSF_mask_filename,\
    iso_grid_filename,clean_kinematics_filename = fit_filenames

## File loading and data preparation
fit_stuff,other_stuff = putil.prepare_fitting(fit_filenames,
    [ndmod,dmod_min,dmod_max], ro,zo,return_other=True)
apogee_effSF_mask,dmap,iso_grid,jkmins,dmods,ds,effsel_grid,apof,\
    allstar_nomask,orbs_nomask = fit_stuff
Rgrid,phigrid,zgrid = effsel_grid

# ## Load the distribution functions
# df_filename = df_dir+'dfs.pkl'
# betas = [0.3,0.8]
# dfs = putil.load_distribution_functions(df_filename, betas)

Loading config file from /home/lane/projects/ges-mass/notebooks/config.txt

Loading APOGEE sel. func. from /geir_data/scr/lane/projects/ges-mass/data/gaia_apogee/apogee_dr16_l33_gaia_edr3/apogee_SF.dat
/home/lane/software/galpy/galpy/util/bovy_coords.py:8: FutureWarning: galpy.util.bovy_coords is being deprecated in favor of galpy.util.coords; all functions in there are the same; please switch to the new import, because the old import will be removed in v1.9
  warnings.warn('galpy.util.bovy_coords is being deprecated in favor of galpy.util.coords; all functions in there are the same; please switch to the new import, because the old import will be removed in v1.9',FutureWarning)


Loading APOGEE eff. sel. func. from /geir_data/scr/lane/projects/ges-mass/data/gaia_apogee/apogee_dr16_l33_gaia_edr3/apogee_effSF_grid_inclArea_-2.0_feh_-0.6.dat

Loading APOGEE eff. sel. func. mask from /geir_data/scr/lane/projects/ges-mass/data/gaia_apogee/apogee_dr16_l33_gaia_edr3/apogee_effSF_grid_mask.npy

### Global Parameters

In [5]:
# %load ../../src/nb_modules/global_fitting_params.py
## general kwargs
verbose = True

## HaloFit kwargs (ordering follows HaloFit.__init__)
# allstar and orbs loaded in prep cell
init = None
init_type = 'ML'
# fit_type provided at runtime
mask_disk = True
mask_halo = True
# densfunc, selec provided at runtime
# effsel, effsel_grid, effsel_mask, dmods loaded in prep cell
nwalkers = 100
nit = int(2e3)
ncut = int(1e3)
# usr_log_prior provided at runtime
n_mass = 2000 # int(nwalkers*(nit-ncut))
int_r_range = [2.,70.]
iso = None # Will read from iso_grid_filename
# iso_filename, jkmins loaded in prep cell
# feh_range provided at runtime
# logg_range loaded in config cell
# fit_dir, gap_dir, ksf_dir loaded in prep cell
# version provided at runtime
# ro, vo, zo loaded in config cell

hf_kwargs = {## HaloFit parameters
             'allstar':allstar_nomask,
             'orbs':orbs_nomask,
             'init':init,
             'init_type':init_type,
             # 'fit_type':fit_type, # provided at runtime
             'mask_disk':mask_disk,
             'mask_halo':mask_halo,
             ## _HaloFit parameters
             # 'densfunc':densfunc, # provided at runtime
             # 'selec':selec, # provided at runtime
             'effsel':apof,
             'effsel_mask':apogee_effSF_mask,
             'effsel_grid':effsel_grid,
             'dmods':dmods,
             'nwalkers':nwalkers,
             'nit':nit,
             'ncut':ncut,
             # 'usr_log_prior':usr_log_prior, # provided at runtime
             'n_mass':n_mass,
             'int_r_range':int_r_range,
             'iso':iso,
             'iso_filename':iso_grid_filename,
             'jkmins':jkmins,
             # 'feh_range':feh_range, # provided at runtime
             'logg_range':logg_range,
             'fit_dir':fit_dir,
             'gap_dir':gap_dir,
             'ksf_dir':ksf_dir,
             # 'version':version, # provided at runtime
             'verbose':verbose,
             'ro':ro,
             'vo':vo,
             'zo':zo}

## pmass.fit() function kwargs
# nprocs set in config file
force_fit = True
mle_init = True
just_mle = False
return_walkers = True
optimizer_method = 'Powell'
mass_int_type = 'spherical_grid'
batch_masses = True
make_ml_aic_bic = True
calculate_masses = True
post_optimization = True
mcmc_diagnostic = True

fit_kwargs = {# 'nprocs':nprocs, # Normally given at runtime 
              'force_fit':force_fit,
              'mle_init':mle_init,
              'just_mle':just_mle,
              'return_walkers':return_walkers,
              'optimizer_method':optimizer_method,
              'mass_int_type':mass_int_type,
              'batch_masses':batch_masses,
              'make_ml_aic_bic':make_ml_aic_bic,
              'calculate_masses':calculate_masses,
              'post_optimization':post_optimization,
              'mcmc_diagnostic':mcmc_diagnostic,
              }

# Fits to GS/E halo data. Use -2 < [Fe/H] < -0.6

In [7]:
fit_type = 'gse'
version = '100w_2e3n_ksf_v2.03'
print('n walkers',nwalkers)
print('n iterations/walker',nit)
print('n cut',ncut)
print('total n samples',nwalkers*(nit-ncut))

n walkers 100
n iterations/walker 2000
n cut 1000
total n samples 100000


## Fits to data with e-Lz selection, GS/E [Fe/H] range

In [11]:
densfunc = pdens.triaxial_single_angle_zvecpa
selec = ['eLz',]

hf = pmass.HaloFit(densfunc=densfunc,
                   selec=selec,
                   fit_type=fit_type,
                   feh_range=feh_range_fit,
                   version=version,
                   **hf_kwargs)

Loading APOGEE kin. eff. sel. grid from /geir_data/scr/lane/projects/ges-mass/data/ksf/apogee_dr16_l33_gaia_edr3/v2_lb_ps_hb_tp/v2.03_beta_03_6e8_09_1.5e8/kSF_grid_linear_eLz.dat
Using informed init
init_type: uninformed
densfunc_source: None


In [12]:
hf.n_star

151

### Triaxial Single power law

In [13]:
# def usr_log_prior(densfunc,params):
#     return 0

# densfunc = pdens.triaxial_single_angle_zvecpa
# hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
# pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

# densfunc = pdens.triaxial_single_angle_zvecpa_plusexpdisk
# hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
# pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

### Triaxial exponential cutoff

In [14]:
def usr_log_prior(densfunc,params):
    assert densfunc is pdens.triaxial_single_cutoff_zvecpa_plusexpdisk\
        or densfunc is pdens.triaxial_single_cutoff_zvecpa
    if params[1] < 2.: return -np.inf
    if params[1] > 55.: return -np.inf
    return 0

# densfunc = pdens.triaxial_single_cutoff_zvecpa
# hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
# pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

densfunc = pdens.triaxial_single_cutoff_zvecpa_plusexpdisk
hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

Setting densfunc to: triaxial_single_cutoff_zvecpa_plusexpdisk
Version is: 100w_2e3n_ksf_v2.03/
Using init_type: ML
init_type: uninformed
densfunc_source: triaxial_single_cutoff_zvecpa
source_fit_data_dir: /geir_data/scr/lane/projects/ges-mass/data/fitting/data/gse/eLz/-2.0_feh_-0.6/triaxial_single_cutoff_zvecpa/100w_2e3n_ksf_v2.03/
Doing maximum likelihood to find starting point
MLE successful
MLE result: [-4.56307076  3.13571712  0.49732774  0.28570361  0.18887144  0.99751152
  0.55987621  0.59285967]
Running MCMC with 10 processers
Generating MCMC samples...
sampled 200/2000 per walker
mean tau: 19.3208492662013
min nit/tau: 9.581719964041103
[min,max] delta tau: [inf,inf]
[min,max] acceptance fraction: [0.22,0.5]
total max nit/tau 9.581719964041103
(nit/tau)/max[(nit/tau)] 1.0
---

sampled 400/2000 per walker
mean tau: 35.350712787316056
min nit/tau: 10.098353250223084
[min,max] delta tau: [-0.4750461091584504,-0.407223424186229]
[min,max] acceptance fraction: [0.24,0.44]
total max

### Triaxial broken power law

In [ ]:
# def usr_log_prior(densfunc,params):
#     assert densfunc is pdens.triaxial_broken_angle_zvecpa_plusexpdisk\
#         or densfunc is pdens.triaxial_broken_angle_zvecpa
#     if params[0] > 10.: return -np.inf
#     if params[1] > 10.: return -np.inf
#     if params[2] < 2.: return -np.inf
#     if params[2] > 55.: return -np.inf
#     return 0

# densfunc = pdens.triaxial_broken_angle_zvecpa
# hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
# pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

# densfunc = pdens.triaxial_broken_angle_zvecpa_plusexpdisk
# hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
# pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

### Triaxial double broken power law

In [ ]:
# def usr_log_prior(densfunc,params):
#     # return 0
#     assert densfunc is pdens.triaxial_double_broken_angle_zvecpa_plusexpdisk\
#         or densfunc is pdens.triaxial_double_broken_angle_zvecpa
#     if params[0] > 10.: return -np.inf
#     if params[1] > 10.: return -np.inf
#     if params[2] > 10.: return -np.inf
#     #if params[3] < 2.: return -np.inf
#     if params[3] > 55.: return -np.inf
#     #if params[4] < 2.: return -np.inf
#     if params[4] > 55.: return -np.inf
#     return 0

# densfunc = pdens.triaxial_double_broken_angle_zvecpa
# hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
# pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

# densfunc = pdens.triaxial_double_broken_angle_zvecpa_plusexpdisk
# hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
# pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

## Fits to data with AD selection, GS/E [Fe/H] range

In [15]:
selec = ['AD',]
densfunc = pdens.triaxial_single_angle_zvecpa
hf = pmass.HaloFit(densfunc=densfunc,
                   selec=selec,
                   fit_type=fit_type,
                   feh_range=feh_range_fit,
                   version=version,
                   **hf_kwargs)

Loading APOGEE kin. eff. sel. grid from /geir_data/scr/lane/projects/ges-mass/data/ksf/apogee_dr16_l33_gaia_edr3/v2_lb_ps_hb_tp/v2.03_beta_03_6e8_09_1.5e8/kSF_grid_linear_AD.dat
Using informed init
init_type: uninformed
densfunc_source: None


### Triaxial single power law

In [ ]:
# def usr_log_prior(densfunc,params):
#     return 0

# densfunc = pdens.triaxial_single_angle_zvecpa
# hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
# pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

# densfunc = pdens.triaxial_single_angle_zvecpa_plusexpdisk
# hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
# pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

### Triaxial exponential cutoff

In [16]:
def usr_log_prior(densfunc,params):
    assert densfunc is pdens.triaxial_single_cutoff_zvecpa_plusexpdisk\
        or densfunc is pdens.triaxial_single_cutoff_zvecpa
    if params[1] < 2.: return -np.inf
    if params[1] > 55.: return -np.inf
    return 0

densfunc = pdens.triaxial_single_cutoff_zvecpa
hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

# densfunc = pdens.triaxial_single_cutoff_zvecpa_plusexpdisk
# hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
# pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

Setting densfunc to: triaxial_single_cutoff_zvecpa
Version is: 100w_2e3n_ksf_v2.03/
Using init_type: ML
init_type: uninformed
densfunc_source: triaxial_single_angle_zvecpa
source_fit_data_dir: /geir_data/scr/lane/projects/ges-mass/data/fitting/data/gse/AD/-2.0_feh_-0.6/triaxial_single_angle_zvecpa/100w_2e3n_ksf_v2.03/
Doing maximum likelihood to find starting point
MLE successful
MLE result: [ 0.27558301 22.81772573  0.29202672  0.10000562  0.3577982   0.98196773
  0.54843432]
Running MCMC with 10 processers
Generating MCMC samples...
sampled 200/2000 per walker
mean tau: 19.16489168434044
min nit/tau: 8.966582306201
[min,max] delta tau: [inf,inf]
[min,max] acceptance fraction: [0.24,0.6]
total max nit/tau 8.966582306201
(nit/tau)/max[(nit/tau)] 1.0
---

sampled 400/2000 per walker
mean tau: 36.5311247964132
min nit/tau: 9.182339707353584
[min,max] delta tau: [-0.525067065210025,-0.4203793990883441]
[min,max] acceptance fraction: [0.26,0.48]
total max nit/tau 9.182339707353584
(nit/tau

### Triaxial broken power law

In [ ]:
# def usr_log_prior(densfunc,params):
#     assert densfunc is pdens.triaxial_broken_angle_zvecpa_plusexpdisk\
#         or densfunc is pdens.triaxial_broken_angle_zvecpa
#     if params[0] > 10.: return -np.inf
#     if params[1] > 10.: return -np.inf
#     if params[2] < 2.: return -np.inf
#     if params[2] > 55.: return -np.inf
#     return 0

# densfunc = pdens.triaxial_broken_angle_zvecpa
# hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
# pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

# densfunc = pdens.triaxial_broken_angle_zvecpa_plusexpdisk
# hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
# pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

### Triaxial double broken power law

In [ ]:
# def usr_log_prior(densfunc,params):
#     # return 0
#     assert densfunc is pdens.triaxial_double_broken_angle_zvecpa_plusexpdisk\
#         or densfunc is pdens.triaxial_double_broken_angle_zvecpa
#     if params[0] > 10.: return -np.inf
#     if params[1] > 10.: return -np.inf
#     if params[2] > 10.: return -np.inf
#     #if params[3] < 2.: return -np.inf
#     if params[3] > 55.: return -np.inf
#     #if params[4] < 2.: return -np.inf
#     if params[4] > 55.: return -np.inf
#     return 0

# densfunc = pdens.triaxial_double_broken_angle_zvecpa
# hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
# pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

# densfunc = pdens.triaxial_double_broken_angle_zvecpa_plusexpdisk
# hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
# pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

## Fits to data with JRLz selection, GS/E [Fe/H] range

In [17]:
selec = ['JRLz',]
densfunc = pdens.triaxial_single_angle_zvecpa
hf = pmass.HaloFit(densfunc=densfunc,
                   selec=selec,
                   fit_type=fit_type,
                   feh_range=feh_range_fit,
                   version=version,
                   **hf_kwargs)

Loading APOGEE kin. eff. sel. grid from /geir_data/scr/lane/projects/ges-mass/data/ksf/apogee_dr16_l33_gaia_edr3/v2_lb_ps_hb_tp/v2.03_beta_03_6e8_09_1.5e8/kSF_grid_linear_JRLz.dat
Using informed init
init_type: uninformed
densfunc_source: None


### Triaxial single power law

In [18]:
def usr_log_prior(densfunc,params):
    return 0

# densfunc = pdens.triaxial_single_angle_zvecpa
# hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
# pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

densfunc = pdens.triaxial_single_angle_zvecpa_plusexpdisk
hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

Setting densfunc to: triaxial_single_angle_zvecpa_plusexpdisk
Version is: 100w_2e3n_ksf_v2.03/
Using init_type: ML
init_type: uninformed
densfunc_source: triaxial_single_angle_zvecpa
source_fit_data_dir: /geir_data/scr/lane/projects/ges-mass/data/fitting/data/gse/JRLz/-2.0_feh_-0.6/triaxial_single_angle_zvecpa/100w_2e3n_ksf_v2.03/
Doing maximum likelihood to find starting point
MLE successful
MLE result: [1.7699179  0.5479147  0.47869057 0.05684719 0.5        0.57646384
 0.6256443 ]
Running MCMC with 10 processers
Generating MCMC samples...
sampled 200/2000 per walker
mean tau: 17.42724372807951
min nit/tau: 9.001123883043386
[min,max] delta tau: [inf,inf]
[min,max] acceptance fraction: [0.27,0.55]
total max nit/tau 9.001123883043386
(nit/tau)/max[(nit/tau)] 1.0
---

sampled 400/2000 per walker
mean tau: 32.36756887056218
min nit/tau: 8.430407808568955
[min,max] delta tau: [-0.5317024897051781,-0.373461377293968]
[min,max] acceptance fraction: [0.29,0.52]
total max nit/tau 9.0011238830

### Triaxial exponential cutoff

In [ ]:
# def usr_log_prior(densfunc,params):
#     assert densfunc is pdens.triaxial_single_cutoff_zvecpa_plusexpdisk\
#         or densfunc is pdens.triaxial_single_cutoff_zvecpa
#     if params[1] < 2.: return -np.inf
#     if params[1] > 55.: return -np.inf
#     return 0

# densfunc = pdens.triaxial_single_cutoff_zvecpa
# hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
# pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

# densfunc = pdens.triaxial_single_cutoff_zvecpa_plusexpdisk
# hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
# pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

### Triaxial broken power law

In [ ]:
# def usr_log_prior(densfunc,params):
#     assert densfunc is pdens.triaxial_broken_angle_zvecpa_plusexpdisk\
#         or densfunc is pdens.triaxial_broken_angle_zvecpa
#     if params[0] > 10.: return -np.inf
#     if params[1] > 10.: return -np.inf
#     if params[2] < 2.: return -np.inf
#     if params[2] > 55.: return -np.inf
#     return 0

# densfunc = pdens.triaxial_broken_angle_zvecpa
# hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
# pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

# densfunc = pdens.triaxial_broken_angle_zvecpa_plusexpdisk
# hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
# pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

### Triaxial double broken power law

In [ ]:
# def usr_log_prior(densfunc,params):
#     # return 0
#     assert densfunc is pdens.triaxial_double_broken_angle_zvecpa_plusexpdisk\
#         or densfunc is pdens.triaxial_double_broken_angle_zvecpa
#     if params[0] > 10.: return -np.inf
#     if params[1] > 10.: return -np.inf
#     if params[2] > 10.: return -np.inf
#     #if params[3] < 2.: return -np.inf
#     if params[3] > 55.: return -np.inf
#     #if params[4] < 2.: return -np.inf
#     if params[4] > 55.: return -np.inf
#     return 0

# densfunc = pdens.triaxial_double_broken_angle_zvecpa
# hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
# pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

# densfunc = pdens.triaxial_double_broken_angle_zvecpa_plusexpdisk
# hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
# pmass.fit(hf, nprocs=nprocs, **fit_kwargs)